# Creation of Validation Set (VS) post Barnacle

Creation of a validation set, with all possible molecules, released after training of barnacle. So we should in general avoid overlap with Barnacle Training data.

In [1]:
# Please execute as header first!

import pandas as pd
filename_csv = "VS_overview.csv"

df = pd.read_csv(filename_csv, comment="#")

Downloading all sequences from corresponding `rcsb.org` id (and manually cured the chain afterwards):

In [52]:
for i, row in df.iterrows():
    ! curl --silent https://www.rcsb.org/fasta/entry/"{row.PDB}"/download > sequences/"{row.PDB}"_"{row.Chain}".fasta

Adding `filename`to csv:

In [53]:
def add_filename(row:pd.Series) -> str:
    return f"{row.PDB}_{row.Chain}"

df["Filename"] = df.apply(add_filename, axis=1)

Adding molecule length `L` to csv:

In [4]:
def get_L_from_file(row: pd.Series) -> int:
    with open(f"sequences/{row.Filename}.fasta", "r") as sequence_file: 
        sequence_file.readline()
        return len(sequence_file.readline().rstrip())
    return 0

df["L"] = df.apply(get_L_from_file, axis=1)

Extracting covariance for each family:

In [56]:
families = df['Family']
for family in families:
    ! cmfetch Rfam.cm "{family}" > cm/"{family}".cm

Downloading all family core sequences from ``rfam.org``:

In [60]:
familes = df["Family"]
for family in families:
    ! curl --silent -o MSA/"{family}".fasta https://rfam.org/family/"{family}"/alignment/fastau

Add molecule sequence as first entry into MSA:

In [61]:
families = df["Family"]
for fam in families:
    with open(f"MSA/{fam}.fasta", "r+") as msa_file:
        # print(fam, df[df["Family"] == fam].Filename.item())
        msa_content = msa_file.read()
        with open(
            f"sequences/{df[df['Family'] == fam].Filename.item()}.fasta", "r"
        ) as sequence_file:
            target_sequence = sequence_file.read().rstrip("\n")
            msa_file.seek(0, 0)
            msa_file.write(target_sequence + "\n" + msa_content)

Align according to first sequence:

In [62]:
families = df["Family"]
for fam in families:
    ! cmalign cm/"{fam}".cm MSA/"{fam}".fasta > MSA/"{fam}".stk
    ! esl-reformat afa MSA/"{fam}".stk > MSA/"{fam}".fa
    ! rm MSA/"{fam}".fasta MSA/"{fam}".stk

Download all PDB Files:

In [ ]:
from Bio.PDB import MMCIFParser, PDBIO

families = df["Family"]
for fam in families:
    pdb_name = df[df['Family'] == fam].PDB.item()
    chain_id = df[df['Family'] == fam].Chain.item()
    print(pdb_name)
    ! curl -f https://files.rcsb.org/download/"{pdb_name}".cif.gz -o ./PDB/"{pdb_name}".cif.gz
    ! gunzip -f ./PDB/"{pdb_name}".cif.gz
    parser = MMCIFParser()
    struc = parser.get_structure(pdb_name, f"PDB/{pdb_name}.cif")
    io = PDBIO()
    io.set_structure(struc[0][chain_id])
    io.save(f"PDB/{pdb_name}_{chain_id}.pdb")

In [72]:
df.to_csv("VS_final.csv", index=False)

## DCA & CoCoNet Prediction

Firstly trim all the MSA files:

In [65]:
families = df["Family"]


with open("cut_files.sh", "w") as f:
    f.write("#!/bin/bash \n")
    for fam in families:
        f.write(
            f"pydca trim_by_refseq 'rna' uncut/{fam}.fa sequences/{df[df['Family'] == fam].Filename.item()}.fasta --remove_all_gaps --verbose \n"
        )

Calculate the effective number of sequences in the MSAs with https://gitlab.jsc.fz-juelich.de/faber1/sequeff:

In [70]:
families = df["Family"]

Meff = []
for family in families:
    result = ! ./sequeff -f MSA/Trimmed_"{family}".fa
    Meff.append(float(result[0]))

df["Meff"] = Meff

Create CoCoNet predictions (including pyDCA):

In [3]:
families = df["Family"]

with open("run_coconet.sh", "w") as f:
    f.write("#!/bin/bash \n")
    for fam in families:
        f.write(f"python -m coconet.main ../cut/Trimmed_{fam}.fa \n")

Create sequence files with only one line as input for SimRNA:

In [4]:
df = pd.read_csv("VS_final.csv", comment="#")

for i, row in df.iterrows():
    with open(f"sequences/{row.Filename}.fasta", "r") as in_file, open(f"sequences/{row.Filename}.fa", "w") as out_file:
        out_file.write(in_file.readlines()[1])